In [1]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [2]:
sample = """<h1>Title Goes Here</h1>
<b>Bolded Text</b>
<i>Italicized Text</i>
<img src="this should all be gone"/>
<a href="this will be gone, too">But this will still be here!</a>
I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?
[Some text we don't want to keep is in here]
¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!
something... is! wrong() with.,; this :: sentence.
I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?
My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.
Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.
[This is some other unwanted text]
John: "Well, well, well."
James: "There, there. There, there."
&nbsp;&nbsp;
There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.
22    45   1067   445
{{Here is some stuff inside of double curly braces.}}
{Here is more stuff in single curly braces.}
[DELETE]
</body>
</html>"""

## Remoção de Ruído

Nesse processo de remoção de ruídos faremos:
 - Remoção do cabeçalho e do rodapé
 - Remoção dos códigos HTML e XML
 - Extração de dados importantes de outros formatos, como JSON

Na célula seguinte é utilizada a biblioteca Beautiful Soup para a remoção das tags HTML, através do parser html que esta possui

Na segunda função é feita a remoção de colchetes duplos ("[[]]") utilizando expressões regulares e remove-se todo texto dentro dos colchetes duplos. Essa remoção é específica deste exemplo.

In [3]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [4]:
sample = denoise_text(sample)

In [5]:
display(sample)

'Title Goes Here\nBolded Text\nItalicized Text\n\nBut this will still be here!\nI run. He ran. She is running. Will they stop running?\nI talked. She was talking. They talked to them about running. Who ran to the talking runner?\n\n¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!\nsomething... is! wrong() with.,; this :: sentence.\nI can\'t do this anymore. I didn\'t know them. Why couldn\'t you have dinner at the restaurant?\nMy favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.\nDon\'t do it.... Just don\'t. Billy! I know what you\'re doing. This is a great little house you\'ve got here.\n\nJohn: "Well, well, well."\nJames: "There, there. There, there."\n\xa0\xa0\nThere are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.\nI have to go get 2 tutus from 2 different stores, too.\n22    45   1067   445\n{{Here is some stuff inside of

Agora iremos remover as contrações. Em inglês, encontramos diversos textos, sendo eles formais ou informais, que possuem contrações como _didn't_ ou _don't_. Ao utilizar um tokenizer, ou seja, ao separar nossas palavras nos passos a frente, essas contrações serão extraídas de forma que irão inserir ruídos em nossos dados. Uma contração como _didn't_ seria transformada em dois tokens ("did" e "n't"). Então para isso, removemos as contrações, transformando em duas palavras diferentes: did e not. Para isso iremos utilizar a biblioteca contractions

In [6]:
def replace_contractions(text):
    return contractions.fix(text)

In [7]:
sample_contractions_fixed = replace_contractions(sample) 

In [8]:
sample_contractions_fixed

'Title Goes Here\nBolded Text\nItalicized Text\n\nBut this will still be here!\nI run. He ran. She is running. Will they stop running?\nI talked. She was talking. They talked to them about running. Who ran to the talking runner?\n\n¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!\nsomething... is! wrong() with.,; this :: sentence.\nI can not do this anymore. I did not know them. Why could not you have dinner at the restaurant?\nMy favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.\ndo not do it.... Just do not. Billy! I know what you are doing. This is a great little house you have got here.\n\nJohn: "Well, well, well."\nJames: "There, there. There, there."\n\xa0\xa0\nThere are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.\nI have to go get 2 tutus from 2 different stores, too.\n22    45   1067   445\n{{Here is some stuff inside 

## Tokenization

Ao finalizar nossa remoção de ruídos, agora iremos para o processo de Tokenization. Basicamente, tokenization é o processo de segmentar uma grande quantidade de texto em elementos menores, sejam esses elementos parágrafos, sentenças ou palavras. Normalmente tokenization se refere à segmentação do texto em palavras, enquanto segmentação refere-se à particionar o texto em elementos maiores que uma palavra. Normalmente um processamento mais profundo do texto é feito apenas após o processo de segmentação.

E para tokenizar é bem simples. Utilizando o NLTK (Natural Language ToolKit) executamos a função word_tokenize na nossa amostra de texto. Lembrando que o NLTK possui alguns downloads adicionais que devem ser feitos para a execução de certas funções, então na execução das funções dessa biblioteca podem ser disparados erros, porém basta apenas ler os erros e verificar qual download deve ser feito.

In [9]:
words = nltk.word_tokenize(sample_contractions_fixed)

In [10]:
len(words)

248

## Normalization

O processo de normalização consiste em uma série de tarefas para colocar todas as palavras em "grupos" similares. Exemplos dessas tarefas podem ser: remoção de pontuação, transformação de maiúsculo para minúsculo, remoção de sufixos e prefixos e assim por diante. Dessa forma, conseguimos reduzir o número de palavras e obter apenas palavras úteis para nosso problema. Nesse notebook iremos focar em 3 processos de normalização.

## Stemming

Stemming consiste na remoção de afixos (sufixos, prefixos, infixos, circunfixos) com o intuito de obter apenas a raiz da palavra. Este processo tem o intuito de reduzir a quantidade de diferentes palavras que apresentam o mesmo significado ou "objetivo". Por exemplo, palavras como _running_ ou _runner_ são convertidas para _run_. Para esse processo iremos utilizar novamente a biblioteca NLTK, com a função LancasterStemmer

In [11]:
def stem_words(words):
    
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

## Lemmatization

O processo de lemmatization é bem mais sofisticado. Esse processo busca extrair a forma canônica das palavras, que vai além de apenas remover afixos. Por exemplo, a palavra melhor é transformada em bom, pior em ruim, corredor em correr, e assim por diante. É um processo parecido com Stemming, porém stemming poderia transformar de formas erradas algumas palavras. Se pegássemos a palavra _better_ e retirássemos apenas seu sufixo ficaríamos com a palavra _bet_, enquanto se aplicamos o lemmatization iremos ficar com a palavra _good_.

In [12]:
def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

## Todo o resto

Devido a este notebook tratar de uma introdução ao pré-processamento de texto e por lemmatization e stemming se tratarem de processos extremamente importantes e com certo grau de sofisticação, se fazendo necessário conhecimento de linguagem para compreendê-los, esses dois métodos principais foram abordados separadamente.

Porém agora vamos lidar com técnicas mais gerais de limpeza do texto, agregando todas elas no mesmo contexto. Lembrando que as técnicas utilizadas a seguir também são essenciais para o resultado futuro, entretanto envolvem operações básicas de substituição e remoção nos textos.

E essas tarefas são:
 - Transformar todas as letras para minúsculas
 - Remover números ou escreve-los por extenso
 - Remover pontuação
 - Remover espaços em branco
 - Remover _stop-words_ padrão

_Stop Words_ são palavras que não têm nenhum valor semântico que possa cooperar para a resolução do problema, como por exemplo: artigos, conjunções, preposições, etc. São palavras que não agregam valor ao modelo e podem se tornar ruído, adicionando mais uma dimensão ao modelo e atrapalhando a classificação. Há também a existência de _stop words_ específicas da tarefa que está sendo executada. Por exemplo, em textos de redes sociais, caso estejamos procurando extrair o conteúdo de um post, seria uma boa alternativa utilizar algumas gírias como _stop words_, como as palavras "tipo" ou "caramba", que irão se repetir constantemente mas que não agregam ao conteúdo do texto.


In [13]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [14]:
words = normalize(words)

In [15]:
len(words)

88

In [16]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

In [19]:
stems, lemmas = stem_and_lemmatize(words)

In [20]:
print(stems)

['titl', 'goe', 'bold', 'text', 'it', 'text', 'stil', 'run', 'ran', 'run', 'stop', 'run', 'talk', 'talk', 'talk', 'run', 'ran', 'talk', 'run', 'sebast', 'nicola', 'alejandro', 'jeronimo', 'going', 'stor', 'tomorrow', 'morn', 'someth', 'wrong', 'sent', 'anym', 'know', 'could', 'din', 'resta', 'favorit', 'movy', 'franch', 'ord', 'indian', 'jon', 'marvel', 'cinem', 'univers', 'star', 'war', 'back', 'fut', 'harry', 'pot', 'bil', 'know', 'gre', 'littl', 'hous', 'got', 'john', 'wel', 'wel', 'wel', 'jam', 'lot', 'reason', 'one hundred and on', 'reason', 'one million', 'reason', 'act', 'go', 'get', 'two', 'tut', 'two', 'diff', 'stor', 'twenty-two', 'forty-five', 'one thousand and sixty-seven', 'four hundred and forty-five', 'stuff', 'insid', 'doubl', 'cur', 'brac', 'stuff', 'singl', 'cur', 'brac']


In [21]:
print(lemmas)

['title', 'go', 'bolded', 'text', 'italicize', 'text', 'still', 'run', 'run', 'run', 'stop', 'run', 'talk', 'talk', 'talk', 'run', 'run', 'talk', 'runner', 'sebastian', 'nicolas', 'alejandro', 'jeronimo', 'go', 'store', 'tomorrow', 'morning', 'something', 'wrong', 'sentence', 'anymore', 'know', 'could', 'dinner', 'restaurant', 'favorite', 'movie', 'franchise', 'order', 'indiana', 'jones', 'marvel', 'cinematic', 'universe', 'star', 'war', 'back', 'future', 'harry', 'potter', 'billy', 'know', 'great', 'little', 'house', 'get', 'john', 'well', 'well', 'well', 'jam', 'lot', 'reason', 'one hundred and one', 'reason', 'one million', 'reason', 'actually', 'go', 'get', 'two', 'tutus', 'two', 'different', 'store', 'twenty-two', 'forty-five', 'one thousand and sixty-seven', 'four hundred and forty-five', 'stuff', 'inside', 'double', 'curly', 'brace', 'stuff', 'single', 'curly', 'brace']
